<a href="https://colab.research.google.com/github/eldarski/microbiology_AI/blob/main/Microscope_Colony_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from google.colab import drive
from tqdm import tqdm
from scipy import ndimage
from concurrent.futures import ThreadPoolExecutor

def correct_illumination(image, block_size=201):
    """
    Correct non-uniform illumination using downsampling method
    """
    # Convert to float for processing
    img_float = image.astype(float)

    # Calculate background on smaller image
    scale_factor = 0.2  # Reduce size to 20%
    small_shape = tuple(int(dim * scale_factor) for dim in image.shape[:2])

    # Downsample
    small_img = cv2.resize(img_float, small_shape[::-1])

    # Ensure kernel size is odd
    kernel_size = max(3, block_size//5)
    if kernel_size % 2 == 0:
        kernel_size += 1

    # Blur small image
    small_blur = cv2.GaussianBlur(small_img, (kernel_size, kernel_size), 0)

    # Upsample back to original size
    background = cv2.resize(small_blur, (image.shape[1], image.shape[0]))

    # Subtract background and normalize
    corrected = img_float - background
    corrected = corrected - corrected.min()
    corrected = (corrected * 255.0 / corrected.max()).astype(np.uint8)

    return corrected

def process_image_in_patches(image, patch_size=1500, overlap=300):
    """Process large image in overlapping patches with optimized memory usage"""
    height, width = image.shape[:2]
    patches = []

    # Increase overlap for smoother merging
    patch_size = min(patch_size, height//2, width//2)
    overlap = min(overlap, patch_size//3)

    # Calculate steps to ensure coverage of entire image
    y_steps = range(0, height-patch_size+overlap, patch_size-overlap)
    x_steps = range(0, width-patch_size+overlap, patch_size-overlap)

    # Add final patches if needed
    if y_steps[-1] + patch_size < height:
        y_steps = list(y_steps) + [height - patch_size]
    if x_steps[-1] + patch_size < width:
        x_steps = list(x_steps) + [width - patch_size]

    total_patches = len(y_steps) * len(x_steps)

    with tqdm(total=total_patches, desc="Planning patches") as pbar:
        for y in y_steps:
            for x in x_steps:
                patches.append((y, y+patch_size, x, x+patch_size))
                pbar.update(1)

    return patches

def create_weight_matrix(patch_size, overlap):
    """Create Gaussian weight matrix for smooth patch blending"""
    y = np.linspace(-1, 1, overlap)
    x = np.linspace(-1, 1, overlap)

    # Create Gaussian weights for edges
    weight_y = np.ones((patch_size, patch_size))
    weight_x = np.ones((patch_size, patch_size))

    # Apply Gaussian feathering to edges
    weight_y[:overlap, :] *= np.exp(-y[:, np.newaxis]**2)
    weight_y[-overlap:, :] *= np.exp(-y[::-1, np.newaxis]**2)
    weight_x[:, :overlap] *= np.exp(-x[np.newaxis, :]**2)
    weight_x[:, -overlap:] *= np.exp(-x[::-1][np.newaxis, :]**2)

    return weight_y * weight_x

def segment_colonies(image_path, min_size=5, max_size=300):
    """Optimized colony segmentation with improved patch handling"""
    print("\nProcessing steps:")

    # Read and preprocess image
    print("1. Loading and preprocessing image...", end=' ', flush=True)
    image = cv2.imread(image_path)
    original = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect petri dish
    print("\n2. Detecting petri dish...", end=' ', flush=True)
    dish_mask = detect_petri_dish(image)
    print("Done")

    # Apply dish mask to gray image
    gray[~dish_mask] = 0

    # Correct illumination inside dish
    print("3. Correcting illumination...", end=' ', flush=True)
    corrected = correct_illumination(gray, block_size=201)
    print("Done")

    # Get patch coordinates
    print("4. Planning patch processing:")
    patch_size = 2000
    overlap = 800  # Further increased overlap
    patches = process_image_in_patches(corrected, patch_size=patch_size, overlap=overlap)

    # Create full-size mask and weight accumulator
    mask = np.zeros_like(gray, dtype=float)
    weight_acc = np.zeros_like(gray, dtype=float)

    # Create Gaussian weight matrix for blending
    weight_matrix = create_weight_matrix(patch_size, overlap)

    # Process patches
    print("5. Processing patches:")
    for y1, y2, x1, x2 in tqdm(patches, desc="Analyzing colonies"):
        # Extract patch
        patch = corrected[y1:y2, x1:x2]
        local_dish_mask = dish_mask[y1:y2, x1:x2]

        if np.sum(local_dish_mask) < 0.1 * patch.size:
            continue

        # Aggressive contrast enhancement
        enhanced = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8)).apply(patch)

        # Fine-tuned denoising
        denoised = cv2.bilateralFilter(enhanced, d=5, sigmaColor=10, sigmaSpace=10)

        # Sensitive adaptive thresholding
        thresh = cv2.adaptiveThreshold(
            denoised, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV, 15, 2
        )

        # Morphological operations to remove noise
        kernel = np.ones((2,2), np.uint8)
        cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

        # Distance transform and watershed to separate connected cells
        dist_transform = cv2.distanceTransform(cleaned, cv2.DIST_L2, 5)
        _, sure_fg = cv2.threshold(dist_transform, 0.2 * dist_transform.max(), 255, 0)
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(cleaned, sure_fg)

        # Marker labelling
        _, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0

        # Apply watershed
        markers = cv2.watershed(cv2.cvtColor(patch, cv2.COLOR_GRAY2BGR), markers)
        patch_mask = np.zeros_like(patch, dtype=float)
        patch_mask[markers > 1] = 1.0

        # Apply Gaussian weight matrix and accumulate
        weighted_mask = patch_mask * weight_matrix
        mask[y1:y2, x1:x2] += weighted_mask
        weight_acc[y1:y2, x1:x2] += weight_matrix

    # Normalize accumulated mask with lower threshold
    mask = np.divide(mask, weight_acc, where=weight_acc > 0)
    mask = (mask > 0.3).astype(np.uint8) * 255

    print("6. Final cleanup...", end=' ', flush=True)
    # Apply dish mask
    mask = cv2.bitwise_and(mask, mask, mask=dish_mask.astype(np.uint8))

    # Lighter final cleanup
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    print("Done")

    return original, mask, corrected, dish_mask

def detect_petri_dish(image):
    """Detect the circular petri dish and create a mask"""
    # Get original dimensions
    original_height, original_width = image.shape[:2]

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image

    # Resize image for faster processing
    max_dimension = 1000
    scale = max_dimension / max(gray.shape)
    resized = cv2.resize(gray, None, fx=scale, fy=scale)

    # Blur the resized image
    blurred = cv2.GaussianBlur(resized, (9, 9), 0)

    # Calculate parameters for resized image
    resized_height, resized_width = blurred.shape[:2]
    min_radius = min(resized_height, resized_width) // 3
    max_radius = max(resized_height, resized_width) // 2

    # Detect circles using Hough transform on resized image
    try:
        circles = cv2.HoughCircles(
            blurred,
            cv2.HOUGH_GRADIENT,
            dp=1,
            minDist=resized_height,
            param1=50,
            param2=30,
            minRadius=min_radius,
            maxRadius=max_radius
        )
    except Exception as e:
        print(f"Warning: Circle detection failed: {e}")
        return np.ones_like(gray, dtype=bool)

    # Create mask for resized image
    mask = np.zeros((resized_height, resized_width), dtype=np.uint8)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center = (i[0], i[1])
            radius = i[2]
            cv2.circle(mask, center, radius, 255, -1)
    else:
        print("Warning: No circles detected, using full image")
        return np.ones_like(gray, dtype=bool)

    # Resize mask back to original size
    mask = cv2.resize(mask, (original_width, original_height))

    return mask > 0

def create_color_mask(mask):
    """Optimized color mask creation with white background"""
    num_labels, labels = cv2.connectedComponents(mask, connectivity=8)

    # Pre-generate all colors at once
    colors = np.ones((num_labels, 3), dtype=np.uint8) * 255
    if num_labels > 1:
        colors[1:] = np.random.randint(50, 255, (num_labels-1, 3))

    # Use vectorized operation
    colored_mask = colors[labels]

    return colored_mask, num_labels - 1

def save_image(path, image, quality=95):
    """Save image using OpenCV with specified quality"""
    # Use PNG for lossless compression
    if path.endswith('.png'):
        cv2.imwrite(path, image, [cv2.IMWRITE_PNG_COMPRESSION, 3])
    elif path.endswith('.jpg') or path.endswith('.jpeg'):
        cv2.imwrite(path, image, [cv2.IMWRITE_JPEG_QUALITY, quality])

def save_high_res_images(original, colored_mask, petri_mask, save_dir, basename):
    """Save high-resolution colored mask and overlay with improved visibility"""
    # Apply petri dish mask with white background
    colored_mask[~petri_mask] = [255, 255, 255]

    # Create brighter overlay
    overlay = original.copy()
    mask_bool = colored_mask.any(axis=2)
    overlay[mask_bool] = cv2.addWeighted(
        overlay[mask_bool], 0.3,
        colored_mask[mask_bool], 0.7,
        0
    )

    # Prepare paths
    colored_path = os.path.join(save_dir, f'colored_{basename}.png')
    overlay_path = os.path.join(save_dir, f'overlay_{basename}.png')

    # Use ThreadPoolExecutor to save images concurrently
    with ThreadPoolExecutor() as executor:
        executor.submit(save_image, colored_path, cv2.cvtColor(colored_mask, cv2.COLOR_RGB2BGR))
        executor.submit(save_image, overlay_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

def create_legend(color_mapping, num_columns=5):
    """Create a legend figure showing colony numbers and their colors"""
    num_colonies = len(color_mapping)
    num_rows = (num_colonies + num_columns - 1) // num_columns

    plt.figure(figsize=(2*num_columns, 2*num_rows))
    for idx, (colony_id, color) in enumerate(color_mapping.items()):
        plt.subplot(num_rows, num_columns, idx + 1)
        plt.fill([0, 1, 1, 0], [0, 0, 1, 1], color=color/255.0)
        plt.title(f'#{colony_id}')
        plt.axis('off')

    plt.tight_layout()
    return plt.gcf()

def visualize_results(original, mask, corrected, dish_mask=None, save_path=None):
    """
    Plot and save visualization with improved visibility
    """
    # Create multicolor mask where each colony has a unique color
    colored_mask, num_colonies = create_color_mask(mask)

    # Apply dish mask if provided
    if dish_mask is not None:
        colored_mask[~dish_mask] = [255, 255, 255]

    # Create overlay with brighter colors
    overlay = original.copy()
    mask_bool = mask > 0
    overlay[mask_bool] = cv2.addWeighted(
        overlay[mask_bool], 0.3,
        colored_mask[mask_bool], 0.7,
        0
    )

    # Create main figure
    plt.figure(figsize=(20, 5))

    # Plot original
    plt.subplot(141)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    # Plot corrected
    plt.subplot(142)
    plt.imshow(corrected, cmap='gray')
    plt.title('Illumination Corrected')
    plt.axis('off')

    # Plot segmentation (multicolored) with white background
    plt.subplot(143)
    plt.imshow(colored_mask)
    plt.title(f'Segmented Colonies (n={num_colonies})')
    plt.axis('off')

    # Plot overlay with brighter colors
    plt.subplot(144)
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title('Overlay')
    plt.axis('off')

    plt.tight_layout()

    # Save
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close()
    else:
        plt.show()

def process_all_images(input_dir, output_dir):
    """Process all images in the input directory"""
    # Create output directories
    os.makedirs(output_dir, exist_ok=True)

    # Get all images
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    if not image_files:
        raise ValueError("No images found in the input directory!")

    for img_file in tqdm(image_files, desc="Processing images"):
        print(f"\nProcessing image: {img_file}")

        # Paths
        input_path = os.path.join(input_dir, img_file)

        # Segment colonies
        original, mask, corrected, dish_mask = segment_colonies(input_path)

        # Create colored mask for full resolution output
        colored_mask, num_colonies = create_color_mask(mask)

        # Save high-resolution colored mask and overlay
        save_high_res_images(
            original=original,
            colored_mask=colored_mask,
            petri_mask=dish_mask,
            save_dir=output_dir,
            basename=img_file
        )

    print("\nProcessing completed successfully!")

def check_setup():
    base_dir = '/content/drive/MyDrive/colony_segmentation'

    return base_dir

def main():
    """Main function for colony segmentation"""
    drive.mount('/content/drive')
    base_dir = check_setup()

    input_dir = os.path.join(base_dir, 'images')
    output_dir = os.path.join(base_dir, 'predictions')

    process_all_images(input_dir, output_dir)
    print("Segmentation completed! Check the 'predictions' folder for results.")

if __name__ == "__main__":
    try:
        main()
    except ValueError as e:
        print(f"\nSetup Error: {str(e)}")
        print("\nPlease fix the issues and run the code again.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Processing images:   0%|          | 0/26 [00:00<?, ?it/s]


Processing image: A-9.jpg

Processing steps:
1. Loading and preprocessing image... 
2. Detecting petri dish... Done
3. Correcting illumination... Done
4. Planning patch processing:



Planning patches: 100%|██████████| 81/81 [00:00<00:00, 176855.09it/s]


5. Processing patches:


...

5. Processing patches:



Analyzing colonies: 100%|██████████| 81/81 [00:29<00:00,  2.74it/s]


6. Final cleanup... Done


Processing images: 100%|██████████| 26/26 [41:46<00:00, 96.41s/it]


Processing completed successfully!
Segmentation completed! Check the 'predictions' folder for results.
